In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from g_over_t import add_elapsed_time_column
import math

FILTERED_COMBINED_DATA_PATH = R"april21govert\combined_filtered_SBand_March.csv"
data=pd.read_csv(FILTERED_COMBINED_DATA_PATH)
data=add_elapsed_time_column(data)
data

,timestamp_posix,timestamp,power,azimuth,elevation,elapsed
0,1.711418e+09,2024-03-26T02:00:55.162320+00:00,-40.65,116.376838,38.014969,0.000000
1,1.711418e+09,2024-03-26T02:00:55.302502+00:00,-40.65,116.376697,37.795585,0.140182
2,1.711418e+09,2024-03-26T02:00:55.444027+00:00,-40.65,116.376556,37.574098,0.281707
3,1.711418e+09,2024-03-26T02:00:55.596320+00:00,-40.65,116.376404,37.335759,0.434000
4,1.711418e+09,2024-03-26T02:00:55.742233+00:00,-40.65,116.376258,37.107405,0.579913
...,...,...,...,...,...,...
27713,1.711422e+09,2024-03-26T03:07:57.377557+00:00,-40.64,225.004000,90.000000,4022.215237
27714,1.711422e+09,2024-03-26T03:07:57.524484+00:00,-40.63,225.004000,90.000000,4022.362164
27715,1.711422e+09,2024-03-26T03:07:57.670675+00:00,-40.69,225.004000,90.000000,4022.508355
27716,1.711422e+09,2024-03-26T03:07:57.814321+00:00,-40.68,225.004000,90.000000,4022.652001


In [36]:
def isvert(
       data:pd.DataFrame,
       index1:int,
       index2:int,
       angle_threshold_degrees=1.0
) -> bool:
    """determines if two points are vertical. Points that do not move are considered in this case to be vertical

    Args:
        data (pd.DataFrame): _description_
        index1 (int): _description_
        index2 (int): _description_
        angle_threshold_degrees (float, optional): _description_. Defaults to 1.0.

    Returns:
        bool: _description_
    """
    row1=data.iloc[index1]
    row2=data.iloc[index2]

    elevation1=row1[f"elevation"]
    elevation2=row2[f"elevation"]
    azimuth1=row1[f"azimuth"]
    azimuth2=row2[f"azimuth"]

    print(f"{azimuth1=}")
    print(f"{azimuth2=}")
    print(f"{elevation1=}")
    print(f"{elevation2=}")

    rise = elevation2 - elevation1
    run = azimuth2 - azimuth1
    if abs(rise)<(1e-9) and abs(run)<(1e-9):
        return True
    print(f"{rise=}")
    print(f"{run=}")

    angle_degrees=abs(math.degrees(math.atan2(rise,run)))

    print(f"{angle_degrees=}")
    if (90-angle_threshold_degrees)<=angle_degrees<=(90+angle_threshold_degrees):
        return True
    else:
        return False
    
    
x=isvert(data,2,3)
print(x)


azimuth1=116.37655597305296
azimuth2=116.37640368008614
elevation1=37.57409782791137
elevation2=37.335759334802624
rise=-0.2383384931087491
run=-0.00015229296681695814
angle_degrees=90.0366107167448
True


In [31]:
X=isvert(data,27000,2)
print(x)

azimuth1=224.994
azimuth2=116.37655597305296
elevation1=86.22683771514892
elevation2=37.57409782791137
rise=-48.652739887237544
run=-108.61744402694704
angle_degrees=155.871080208954
False
